In [1]:
import os
import cv2
import numpy as np
import skimage as ski
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from preprocess import *
from FeatureExtraction.lpq import lpq 
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [11]:
def extract_hog_features(image):
    # Convert image to grayscale
    # image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Compute HOG features
    features = hog(image, orientations=9, pixels_per_cell=(16, 8), cells_per_block=(2,2)) #orig was cells_per_block=(2,2)
    return features

In [2]:
def get_glcm_features(img):
    # 8-Neighbourhood angles 
    comatrix = ski.feature.graycomatrix(img, distances=[1,2], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256, symmetric=True, normed=True)
    contrast=ski.feature.graycoprops(comatrix, 'contrast')
    dissimilarity=ski.feature.graycoprops(comatrix, 'dissimilarity')
    homogeneity=ski.feature.graycoprops(comatrix, 'homogeneity')
    energy=ski.feature.graycoprops(comatrix, 'energy')
    correlation=ski.feature.graycoprops(comatrix, 'correlation')
    Toreturn=np.concatenate([contrast.ravel(), dissimilarity.ravel(), homogeneity.ravel(), energy.ravel(), correlation.ravel()])
    return Toreturn

In [2]:
def extract_lbp_features(img):
    return ski.feature.local_binary_pattern(img, P=8, R=1).ravel()

In [11]:
dataset_dir = "outresize500meanLoG"

# List to store image paths and corresponding labels
# image_paths = []
labels = []
hog_features=[]
lpq_features=[]
from tqdm import tqdm
# Iterate through each font type directory
for i,font_type in enumerate(os.listdir(dataset_dir)):
    print("Processing font type ",font_type)
    counter=0
    font_type_dir = os.path.join(dataset_dir, font_type)
    font_label=i
    if os.path.isdir(font_type_dir):
        # Iterate through each image in the font type directory
        for image_file in tqdm (os.listdir(font_type_dir)):
            # counter+=1
            # if counter>1000:
            #     break;
            image_path = os.path.join(font_type_dir, image_file)
            # Read the image
            labels.append(font_label)
            # image = io.imread(image_path)
            # if image is not None:
            #     # # Extract HOG features
            #     # currHoG=extract_hog_features(image)
            #     # hog_features.append(currHoG)
            #     # # Extract LPQ features
            #     # currLPQ=lpq(image,1)
            #     currLPQ=extract_lbp_features(image)
            #     lpq_features.append(currLPQ)
            #     labels.append(font_label)
#convert hog_features to float 32
# hog_features=np.array(hog_features).astype(np.float32)

In [5]:
# get PCA 
from sklearn.decomposition import PCA
pca = PCA(n_components=1)
pca.fit(lpq_features)
hog_features_pca = pca.transform(lpq_features)

In [6]:
import pickle
with open('lbp_features_resize500_LoG_.pkl', 'wb') as f:
    pickle.dump(hog_features_pca, f)

In [9]:
hog_features_pca.shape

(4000, 1)

In [25]:
len(labels)

4000

In [7]:
#save pca model
import pickle
filename = 'pca_model_resize500_LBP_LoG_pca99.sav'
pickle.dump(pca, open(filename, 'wb'))

In [8]:
# save in pkl 
import pickle
with open('LBP_features_resize500_LoG_16x8_pca99.pkl', 'wb') as f:
    pickle.dump(hog_features_pca, f)

In [36]:
#load pca
import pickle
filename = 'pca_model_crop500_LoG_pca99.sav'
mypca = pickle.load(open(filename, 'rb'))

# initialize an np array of size 65880
mytestVec = np.random.random((65880,1))
#use pca to transform hog_features
test_pca = mypca.transform(mytestVec.reshape(1,-1))
print(test_pca.shape)

(1, 3)


In [10]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(hog_features_pca, labels, test_size=0.2, random_state=36)

ValueError: Found input variables with inconsistent numbers of samples: [4000, 0]

In [9]:
# Convert lists to numpy arrays
X_train = np.array(X_train)
X_test = np.array(X_test)
# Train SVM model
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

In [ ]:

# Predict on the test set
y_pred = svm_model.predict(X_test)
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)
# get the indices of the misclassified samples
misclassified_idx = np.where(y_pred != y_test)


Accuracy: 0.32625


In [35]:
import joblib

# Save the trained SVM model to a file
joblib.dump(svm_model, 'svm_model_linear_resize500_pca90.pkl')


#loaded_model = joblib.load('svm_model.pkl')

['svm_model_linear_resize500_pca90.pkl']

In [49]:
#rbf
svm_model = SVC(kernel='rbf')
svm_model.fit(X_train, y_train)

SVC()

In [50]:
y_pred = svm_model.predict(X_test)
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.3375


In [26]:
# import grid search
from sklearn.model_selection import GridSearchCV
params_grid = {'C': [2, 5, 7, 10, 12],
          'gamma': [0.001, 0.002, 0.0005, 0.004],
          'kernel':['sigmoid'] }

#Create the GridSearchCV object
grid_clf = GridSearchCV(SVC(class_weight='balanced'), params_grid,verbose=3,refit=True)

#Fit the data with the best possible parameters
grid_clf = grid_clf.fit(X_train, y_train)

#Print the best estimator with it's parameters
print(grid_clf.best_estimator_)


Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV 1/5] END ..C=2, gamma=0.001, kernel=sigmoid;, score=0.620 total time=   7.7s
[CV 2/5] END ..C=2, gamma=0.001, kernel=sigmoid;, score=0.603 total time=   7.9s
[CV 3/5] END ..C=2, gamma=0.001, kernel=sigmoid;, score=0.641 total time=   7.6s
[CV 4/5] END ..C=2, gamma=0.001, kernel=sigmoid;, score=0.603 total time=   7.7s
[CV 5/5] END ..C=2, gamma=0.001, kernel=sigmoid;, score=0.628 total time=   7.9s
[CV 1/5] END ..C=2, gamma=0.002, kernel=sigmoid;, score=0.452 total time=   6.5s
[CV 2/5] END ..C=2, gamma=0.002, kernel=sigmoid;, score=0.411 total time=   6.6s
[CV 3/5] END ..C=2, gamma=0.002, kernel=sigmoid;, score=0.461 total time=   6.7s
[CV 4/5] END ..C=2, gamma=0.002, kernel=sigmoid;, score=0.409 total time=   7.6s
[CV 5/5] END ..C=2, gamma=0.002, kernel=sigmoid;, score=0.416 total time=   6.5s
[CV 1/5] END .C=2, gamma=0.0005, kernel=sigmoid;, score=0.639 total time=   8.5s
[CV 2/5] END .C=2, gamma=0.0005, kernel=sigmoid

KeyboardInterrupt: 